<a href="https://colab.research.google.com/github/caltdreamer/GNN/blob/main/Another_copy_of_Most_recent_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q torch-geometric
!pip install umap-learn
!git clone https://github.com/msesia/arc.git
!pip install conditionalconformal
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install -r /content/conformalized-gnn-master/requirements.txt
%cd /content/conformalized-gnn-master
!mkdir -p /content/conformalized-gnn-master/model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
/content/conformalized-gnn-master


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import NELL
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import CoraFull
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
from tqdm import tqdm
from torch_geometric.nn import SAGEConv, BatchNorm
import arc
from arc import models
from arc import methods
from arc import black_boxes
from arc import others
from arc import coverage
import torch.optim as optim
import numpy as np
import os.path as osp
import pandas as pd
import numpy as np
import collections
import random
import networkx as nx
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from numpy.ma.core import maximum
from conditionalconformal.synthetic_data import generate_cqr_data, indicator_matrix
from conditionalconformal import CondConf

from scipy.stats import norm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
import torch.nn as nn
import copy
import numpy as np

from torch_geometric.nn import GCNConv, GATConv, SAGEConv, SGConv



In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def wsc(X, y, S, delta=0.1, M=1000, random_state=2020, verbose=False):
    rng = np.random.default_rng(random_state)

    def wsc_v(X, y, S, delta, v):
        #print(X.shape)
        #print(y.shape)
        #print(len(S))
        n = len(y)
        cover = np.array([y[i] in S[i] for i in range(n)])
        #print(f'length is {n}')
        #print(n)
        #print(f'coverage is {cover.sum()}')
        z = np.dot(X,v)
        # Compute mass
        z_order = np.argsort(z)
        z_sorted = z[z_order]
        cover_ordered = cover[z_order]
        ai_max = int(np.round((1.0-delta)*n))
        ai_best = 0
        bi_best = n-1
        cover_min = 1
        for ai in np.arange(0, ai_max):
            bi_min = np.minimum(ai+int(np.round(delta*n)),n)
            coverage = np.cumsum(cover_ordered[ai:n]) / np.arange(1,n-ai+1)
            coverage[np.arange(0,bi_min-ai)]=1
            bi_star = ai+np.argmin(coverage)
            cover_star = coverage[bi_star-ai]
            if cover_star < cover_min:
                ai_best = ai
                bi_best = bi_star
                cover_min = cover_star
        #print(cover_min)
        #print(ai_best)
        #print(bi_best)
        return cover_min, z_sorted[ai_best], z_sorted[bi_best]

    def sample_sphere(n, p):
        v = rng.normal(size=(p, n))
        v /= np.linalg.norm(v, axis=0)
        return v.T
    V = sample_sphere(M, p=X.shape[1])

    wsc_list = [[]] * M
    a_list = [[]] * M
    b_list = [[]] * M
    #print('wqwqqw')
    if verbose:
        for m in tqdm(range(M)):
            wsc_list[m], a_list[m], b_list[m] = wsc_v(X, y, S, delta, V[m])
    else:
        #print('iehkwd')
        for m in range(M):
            #print(m)
            #print(X.shape)
            #print(y.shape)
            wsc_list[m], a_list[m], b_list[m] = wsc_v(X, y, S, delta, V[m])
            #print(m)
    #print('ikfrk')
    idx_star = np.argmin(np.array(wsc_list))
    a_star = a_list[idx_star]
    b_star = b_list[idx_star]
    v_star = V[idx_star]
    wsc_star = wsc_list[idx_star]
    return wsc_star, v_star, a_star, b_star

def wsc_unbiased(X, y, S, delta=0.1, M=1000, test_size=0.75, random_state=2020, verbose=False):
    #print('iqhd')
    def wsc_vab(X, y, S, v, a, b):
        n = len(y)
        cover = np.array([y[i] in S[i] for i in range(n)])
        z = np.dot(X,v)
        idx = np.where((z>=a)*(z<=b))
        coverage = np.mean(cover[idx])
        return coverage
    #print('skjdh')

    max_attempts = 5000
    for attempt in range(max_attempts):
        X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X, y, S, test_size=0.75, random_state=attempt)
        cover = np.array([y_train[i] in S_train[i] for i in range(len(y_train))])
        if not all(cover):
            break

    if all(cover):
        print('May cause problem')
    #print(len(y_train))
    #print(cover)
    #print('dasytrdv')
    #print(X_train.shape)
    #print(X_test.shape)
    ##print(y_train.shape)
    #print(y_test.shape)
    #print(len(S_train))
    #print(len(S_test))
    #print(S_train)
    # Find adversarial parameters
    wsc_star, v_star, a_star, b_star = wsc(X_train, y_train, S_train, delta=delta, M=M, random_state=random_state, verbose=verbose)
    #print('ewtddvzdas')
    #print(v_star)
    #print(a_star)
    #print(b_star)
    # Estimate coverage
    coverage = wsc_vab(X_test, y_test, S_test, v_star, a_star, b_star)
    return coverage


Check the results

In [23]:
def compute_weighted_average_coverage(coverages, X_test, f):

    weighted_coverages = [coverages[i] * f(X_test[i]) for i in range(len(X_test))]


    total_weight = sum(f(X_test[i]) for i in range(len(X_test)))


    weighted_average_coverage = sum(weighted_coverages) / total_weight

    return weighted_average_coverage
def create_intervals(min_value, max_value, n):
    interval_length = (max_value - min_value) / n

    # Create the intervals
    intervals = [(min_value + i * interval_length, min_value + (i + 1) * interval_length) for i in range(n)]

    return intervals

In [34]:
import pickle

file_path = '/content/drive/MyDrive/Regression_GNN_results.txt'

with open(file_path, 'rb') as file:
    results = pickle.load(file)

In [ ]:
"""
dataset_results[dataset_name][f'Iteration_{z}'] = {
            'Marginal_Coverage': marg_coverage,
            'current_test_features':current_test_features,
            'coverage_flag':coverage_flag,
            'Average length':average_length,
            'current_data_features':current_data_features,
            'current_calib_test_mask':current_calib_test_mask,
            'current_test_mask_outof_calibtest':current_test_mask_outof_calibtest,
            'current_test_labels':current_test_labels}

"""

In [ ]:

datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012']
#dataset_results = {dataset: {'method_split_conformal': {}} for dataset in datasets}
num_nodes_dataset = my_dict = {
    'Anaheim': 914,
    'ChicagoSketch': 2176 ,
    'county_election_2016': 3234,
    'county_education_2012': 3234,
    'county_income_2012': 3234,
    'county_unemployment_2012':3234,
    'twitch_PTBR':1912
}
weighted_coverage = {dataset: {'Weighted_coverage_density': []} for dataset in datasets}
for dataset in datasets:
    print(dataset)
    with open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_edge_list.txt', 'r') as f:
      edges = [list(map(int, line.strip().split())) for line in f]
    edges_tensor = torch.tensor(edges, dtype=torch.long)
    edge_index = edges_tensor.t().contiguous()
    features = torch.tensor([list(map(float, line.strip().split())) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_features.txt')], dtype=torch.float)
    labels = torch.tensor([float(line.strip()) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_labels.txt')], dtype=torch.float)
    num_nodes = num_nodes_dataset.get(f'{dataset}')
    valid_edges_mask = (edge_index >= 0).all(dim=0) & (edge_index < num_nodes).all(dim=0)
    filtered_edge_index = edge_index[:, valid_edges_mask]

    data = Data(x=features, edge_index=filtered_edge_index, y=labels)
    rank = np.linalg.matrix_rank(data.x.detach().cpu().numpy())
    print(data)
    data_4d = data.x
    scaler = StandardScaler()
    data_4d_std = scaler.fit_transform(data_4d)
    pca = PCA(n_components=1)
    data_reduced = pca.fit_transform(data_4d_std)
    kde = gaussian_kde(data_reduced.T)

    total_coverage = 0
    total_run=30
    for o in range(total_run):

      coverages = results.get(f'{dataset}').get(f'Iteration_{o}').get('coverage_flag')
      X_test = results.get(f'{dataset}').get(f'Iteration_{o}').get('current_test_features')
      X_test_transformed = pca.transform(X_test)
      #print(kde(X_test_transformed[0]))
      # Apply KDE to the PCA-transformed test points
      weighted_coverages = [coverages[i] * kde(X_test_transformed[i:i+1]) for i in range(len(X_test_transformed))]
      total_weight = sum(kde(X_test_transformed[i:i+1]) for i in range(len(X_test_transformed)))

      weighted_average_coverage = sum(weighted_coverages) / total_weight
      #print(weighted_average_coverage)
      total_coverage += weighted_average_coverage
    weighted_coverage[dataset]['Weighted_coverage_density'].append(total_coverage/total_run)






Anaheim
Data(x=[914, 4], edge_index=[2, 3870], y=[914])
ChicagoSketch
Data(x=[2176, 4], edge_index=[2, 15099], y=[2176])
county_election_2016
Data(x=[3234, 6], edge_index=[2, 12715], y=[3234])
county_education_2012
Data(x=[3234, 6], edge_index=[2, 12715], y=[3234])
county_income_2012
Data(x=[3234, 6], edge_index=[2, 12715], y=[3234])
county_unemployment_2012
Data(x=[3234, 6], edge_index=[2, 12715], y=[3234])


In [ ]:
import numpy as np

datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012']
dataset_results = {dataset: {'method_split_conformal': {}} for dataset in datasets}
num_nodes_dataset = my_dict = {
    'Anaheim': 914,
    'ChicagoSketch': 2176 ,
    'county_election_2016': 3234,
    'county_education_2012': 3234,
    'county_income_2012': 3234,
    'county_unemployment_2012':3234,
    'twitch_PTBR':1912
}

weighted_coverage = {dataset: {'Weighted_coverage_density': {}} for dataset in datasets}
for dataset in datasets:
    print(dataset)
    with open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_edge_list.txt', 'r') as f:
      edges = [list(map(int, line.strip().split())) for line in f]
    edges_tensor = torch.tensor(edges, dtype=torch.long)
    edge_index = edges_tensor.t().contiguous()
    features = torch.tensor([list(map(float, line.strip().split())) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_features.txt')], dtype=torch.float)
    labels = torch.tensor([float(line.strip()) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_labels.txt')], dtype=torch.float)
    num_nodes = num_nodes_dataset.get(f'{dataset}')
    valid_edges_mask = (edge_index >= 0).all(dim=0) & (edge_index < num_nodes).all(dim=0)
    filtered_edge_index = edge_index[:, valid_edges_mask]

    data = Data(x=features, edge_index=filtered_edge_index, y=labels)
    rank = np.linalg.matrix_rank(data.x.detach().cpu().numpy())
    print(data)
    data_4d = data.x
    scaler = StandardScaler()
    data_4d_std = scaler.fit_transform(data_4d)
    pca = PCA(n_components=1)
    data_reduced = pca.fit_transform(data_4d_std)
    kde = gaussian_kde(data_reduced.T)

    covariates_np = data.x.detach().cpu().numpy()

    min_value = np.min(covariates_np[:, 2])
    max_value = np.max(covariates_np[:, 2])
    def indicator_function(x, interval):
        return 1 if interval[0] <= x < interval[1] else 0
    intervals = create_intervals(min_value,max_value,10)

    interval_coverage = {i: [] for i in range(10)}
    for o in range(50):
        coverages = results[dataset]['method_split_conformal']['Iteration_{}'.format(o)]['coverage_flag']
        X_test = data.x[results[dataset]['method_split_conformal']['Iteration_{}'.format(o)]['test_mask']]

        for b in range(10):
            weighted_coverages = [coverages[i] * indicator_function(X_test[i:i+1][0][2], intervals[b]) for i in range(len(X_test))]
            total_weight = sum(indicator_function(X_test[i:i+1][0][2], intervals[b]) for i in range(len(X_test)))


            weighted_average_coverage = sum(weighted_coverages) / total_weight if total_weight != 0 else 0
            interval_coverage[b].append(weighted_average_coverage)

    for b in range(10):
        weighted_coverage[dataset]['Weighted_coverage_density'][intervals[b]] = np.mean(interval_coverage[b])



In [35]:
datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012']
for data in datasets:
    #weighted_cov = weighted_coverage[data]['Weighted_coverage_density']
    marginal_cov = np.mean([results[data]['Iteration_{}'.format(i)]['Marginal_Coverage'] for i in range(30)])
    conditional_cov = np.mean([results[data]['Iteration_{}'.format(i)]['conditional_coverage'] for i in range(30)])
    length  = np.mean([results[data]['Iteration_{}'.format(i)]['Average length'] for i in range(30)])

    print(f"{data}:\n"
          f"  - Average length: {length}\n"
          #f"  - Weighted coverage: {weighted_cov}\n"
          f"  - Marginal Coverage: {marginal_cov}\n"
          f"  - Conditional coverage: {conditional_cov}")


Anaheim:
  - Average length: 2.831768751144409
  - Marginal Coverage: 0.9577413479052824
  - Conditional coverage: 0.9448366972826109
ChicagoSketch:
  - Average length: 2.407207489013672
  - Marginal Coverage: 0.9525993883792049
  - Conditional coverage: 0.9293686194050382
county_election_2016:
  - Average length: 1.015498399734497
  - Marginal Coverage: 0.944152498712004
  - Conditional coverage: 0.9156506211909357
county_education_2012:
  - Average length: 2.6729772090911865
  - Marginal Coverage: 0.9525502318392582
  - Conditional coverage: 0.9210911267905411
county_income_2012:
  - Average length: 2.540382146835327
  - Marginal Coverage: 0.9481710458526533
  - Conditional coverage: 0.9287074888153933
county_unemployment_2012:
  - Average length: 3.302426338195801
  - Marginal Coverage: 0.9502833590932508
  - Conditional coverage: 0.899479029393077


In [31]:
datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012']
for data in datasets:
    #weighted_cov = weighted_coverage[data]['Weighted_coverage_density']
    marginal_cov = np.mean([results[data]['method_split_conformal']['Iteration_{}'.format(i)]['Marginal_Coverage'] for i in range(30)])
    conditional_cov = np.mean([results[data]['method_split_conformal']['Iteration_{}'.format(i)]['Conditional_Coverage'] for i in range(30)])
    length  = np.mean([results[data]['method_split_conformal']['Iteration_{}'.format(i)]['Average length'] for i in range(30)])

    print(f"{data}:\n"
          f"  - Marginal coverage: {marginal_cov}\n"
          #f"  - Weighted coverage: {weighted_cov}\n"
          f"  - Length: {length}\n"
          f"  - Conditional coverage: {conditional_cov}")

Anaheim:
  - Marginal coverage: 0.9553527980535281
  - Length: 2.747343723987089
  - Conditional coverage: 0.8925062519562598
ChicagoSketch:
  - Marginal coverage: 0.9519142419601837
  - Length: 2.97746501993987
  - Conditional coverage: 0.8383933823296285
county_election_2016:
  - Marginal coverage: 0.9527835051546389
  - Length: 1.0393593447154383
  - Conditional coverage: 0.8953917418366503
county_education_2012:
  - Marginal coverage: 0.9507216494845359
  - Length: 2.6378783461365076
  - Conditional coverage: 0.8326155520793985
county_income_2012:
  - Marginal coverage: 0.9507903780068727
  - Length: 2.7768732245725025
  - Conditional coverage: 0.7459627927097983
county_unemployment_2012:
  - Marginal coverage: 0.9504123711340201
  - Length: 3.051837706715993
  - Conditional coverage: 0.8614182317642717


In [29]:
datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012']
for data in datasets:
    #weighted_cov = weighted_coverage[data]['Weighted_coverage_density']
    marginal_cov = np.mean([results[data]['method_condition_calibrated']['Iteration_{}'.format(i)]['Marginal_Coverage'] for i in range(30)])
    conditional_cov = np.mean([results[data]['method_condition_calibrated']['Iteration_{}'.format(i)]['Conditional_Coverage'] for i in range(30)])
    length  = np.mean([results[data]['method_condition_calibrated']['Iteration_{}'.format(i)]['Average length'] for i in range(30)])

    print(f"{data}:\n"
          f"  - Marginal coverage: {marginal_cov}\n"
          #f"  - Weighted coverage: {weighted_cov}\n"
          f"  - Length: {length}\n"
          f"  - Conditional coverage: {conditional_cov}")

Anaheim:
  - Marginal coverage: 0.9535279805352798
  - Length: 2.5260525351585885
  - Conditional coverage: 0.8537981286110646
ChicagoSketch:
  - Marginal coverage: 0.9527820316488004
  - Length: 2.866116881406764
  - Conditional coverage: 0.8761236071673199
county_election_2016:
  - Marginal coverage: 0.9519243986254293
  - Length: 1.0090525463011493
  - Conditional coverage: 0.9273379925625475
county_education_2012:
  - Marginal coverage: 0.952508591065292
  - Length: 2.5652335697284907
  - Conditional coverage: 0.9176847692416028
county_income_2012:
  - Marginal coverage: 0.9546391752577319
  - Length: 2.6630490465534598
  - Conditional coverage: 0.9347123512127646
county_unemployment_2012:
  - Marginal coverage: 0.9543986254295531
  - Length: 3.3552544362778556
  - Conditional coverage: 0.884627654021097


In [ ]:
# Print the results
for dataset in datasets:
    print(f"{dataset}:")
    for interval, avg_coverage in weighted_coverage[dataset]['Weighted_coverage_density'].items():
        print(f"  Interval {interval}: Average Weighted Coverage = {avg_coverage}")

In [ ]:
dataset_names = datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012','twitch_PTBR']
num_nodes_dataset = my_dict = {
    'Anaheim': 914,
    'ChicagoSketch': 2176 ,
    'county_election_2016': 3234,
    'county_education_2012': 3234,
    'county_income_2012': 3234,
    'county_unemployment_2012':3234,
    'twitch_PTBR':1912
}
for dataset in dataset_names:

    with open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_edge_list.txt', 'r') as f:
        edges = [list(map(int, line.strip().split())) for line in f]
    edges_tensor = torch.tensor(edges, dtype=torch.long)
    edge_index = edges_tensor.t().contiguous()
    features = torch.tensor([list(map(float, line.strip().split())) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_features.txt')], dtype=torch.float)
    labels = torch.tensor([float(line.strip()) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_labels.txt')], dtype=torch.float)
    num_nodes = num_nodes_dataset.get(f'{dataset}')
    valid_edges_mask = (edge_index >= 0).all(dim=0) & (edge_index < num_nodes).all(dim=0)
    filtered_edge_index = edge_index[:, valid_edges_mask]

    data = Data(x=features, edge_index=filtered_edge_index, y=labels)
    rank = np.linalg.matrix_rank(data.x.detach().cpu().numpy())
    print(f'{dataset} dataset has original feature dimension = {data.num_node_features}')
    print(f'{dataset} dataset has feature rank = {rank}')


Using our methods

In [41]:
datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012']
from tqdm import tqdm
dataset_results = {dataset: {'method_split_conformal': {}, 'method_condition_calibrated': {}} for dataset in datasets}
num_nodes_dataset = my_dict = {
    'Anaheim': 914,
    'ChicagoSketch': 2176 ,
    'county_election_2016': 3234,
    'county_education_2012': 3234,
    'county_income_2012': 3234,
    'county_unemployment_2012':3234,
    'twitch_PTBR':1912
}
for dataset in datasets:
    print(dataset)
    with open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_edge_list.txt', 'r') as f:
      edges = [list(map(int, line.strip().split())) for line in f]
    edges_tensor = torch.tensor(edges, dtype=torch.long)
    edge_index = edges_tensor.t().contiguous()
    features = torch.tensor([list(map(float, line.strip().split())) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_features.txt')], dtype=torch.float)
    labels = torch.tensor([float(line.strip()) for line in open(f'/content/conformalized-gnn-master/dataset_regression/{dataset}_labels.txt')], dtype=torch.float)
    num_nodes = num_nodes_dataset.get(f'{dataset}')
    valid_edges_mask = (edge_index >= 0).all(dim=0) & (edge_index < num_nodes).all(dim=0)
    filtered_edge_index = edge_index[:, valid_edges_mask]

    data = Data(x=features, edge_index=filtered_edge_index, y=labels)
    rank = np.linalg.matrix_rank(data.x.detach().cpu().numpy())

    data_4d = data.x
    scaler = StandardScaler()
    data_4d_std = scaler.fit_transform(data_4d)
    pca = PCA(n_components=1)
    data_reduced = pca.fit_transform(data_4d_std)
    kde = gaussian_kde(data_reduced.T)
    print(f'dataset name {dataset}')
    print(data)

    def create_kdes(data_x):
      kdes = []
      if(dataset!='ChicagoSketch'):
        print('Not Chicago Sketch')
        for i in range(data_x.shape[1]):
            kde = gaussian_kde(data_x[:, i])
            kdes.append(kde)
        return kdes
      else:
        for i in range(data_x.shape[1]-1):
            kde = gaussian_kde(data_x[:,i])
            kdes.append(kde)
        return kdes

    kdes = create_kdes(data.x)


    for c in range(30):

      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      x = data.x.to(device)
      y = data.y.squeeze().to(device)
      train_percentage = 0.4
      test_percentage = 0.5
      split_marginal = []
      split_wsc_condition = []
      split_length = []
      condition_calibrated_marginal = []
      condition_calibrated_wsc_condition = []
      condition_calibrated_length = []


      #Randomly choose train and calitest set

      num_nodes = data.num_nodes
      all_node_indices = list(range(num_nodes))
      random.shuffle(all_node_indices)
      num_train_nodes = int(train_percentage * num_nodes)
      num_testandcali_nodes = num_nodes - num_train_nodes
      train_node_indices = all_node_indices[:num_train_nodes]
      testandcali_node_indices = all_node_indices[num_train_nodes:]

      test_node_indices = testandcali_node_indices[:int(test_percentage * num_testandcali_nodes)]
      cali_node_indices = testandcali_node_indices[int(test_percentage * num_testandcali_nodes):]


      train_mask = torch.zeros(num_nodes, dtype=torch.bool)
      train_mask[train_node_indices] = True
      train_mask = torch.zeros(num_nodes, dtype=torch.bool)
      train_mask[train_node_indices] = True

      testandcali_mask = torch.zeros(num_nodes,dtype=torch.bool)
      testandcali_mask[testandcali_node_indices] = True

      test_mask = torch.zeros(num_nodes,dtype=torch.bool)
      test_mask[test_node_indices] = True


      cali_mask = torch.zeros(num_nodes,dtype=torch.bool)
      cali_mask[cali_node_indices] = True
      #test_mask_lists.append(test_mask)

      #*********************************************
      n_train =train_mask.sum()
      n_calib = test_mask.sum()
      n_test = cali_mask.sum()


      #*********************************************
      #Use same Notation as the synthetic data
      x_train_final, y_train_final, x_calib, y_calib, x_test, y_test= x[train_mask].detach().cpu().numpy(), y[train_mask].detach().cpu().numpy(), x[cali_mask].detach().cpu().numpy(), y[cali_mask].detach().cpu().numpy(), x[test_mask].detach().cpu().numpy(), y[test_mask].detach().cpu().numpy()

      x_all = np.concatenate((x_train_final, x_calib, x_test), axis=0)
      y_all = np.concatenate((y_train_final,y_calib,y_test),axis = 0)
      #*********************************************
      train_loader = NeighborSampler(
          data.edge_index, node_idx=(train_mask),
          sizes=[3, 3, 3], batch_size=32, shuffle=False,
      )
      #*********************************************
      class SAGERegressor(torch.nn.Module):
          def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3):
              super(SAGERegressor, self).__init__()

              self.num_layers = num_layers
              self.convs = torch.nn.ModuleList()
              self.batch_norms = torch.nn.ModuleList()
              self.convs.append(SAGEConv(in_channels, hidden_channels))
              self.batch_norms.append(BatchNorm(hidden_channels))

              for i in range(num_layers - 2):
                  self.convs.append(SAGEConv(hidden_channels, hidden_channels))
                  self.batch_norms.append(BatchNorm(hidden_channels))

              self.convs.append(SAGEConv(hidden_channels, out_channels))
              self.batch_norms.append(BatchNorm(out_channels))
              self.linear1 = torch.nn.Linear(out_channels, hidden_channels)
              self.linear2 = torch.nn.Linear(hidden_channels, 1)

          def reset_parameters(self):

              for conv in self.convs:
                  conv.reset_parameters()
              for bn in self.batch_norms:
                  bn.reset_parameters()
              self.linear1.reset_parameters()
              self.linear2.reset_parameters()

          def forward(self, x, adjs):
              for i, (edge_index, _, size) in enumerate(adjs):
                  x_target = x[:size[1]]
                  x = self.convs[i]((x, x_target), edge_index)

                  x = self.batch_norms[i](x)
                  if i != self.num_layers - 1:
                      x = F.leaky_relu(x, 0.2)
                      x = F.dropout(x, p=0.1, training=self.training)

              x = F.leaky_relu(self.linear1(x), 0.2)
              x = self.linear2(x)

              return x
      class GNN(torch.nn.Module):
            def __init__(self, in_channels, hidden_channels, out_channels, backbone, heads = 1, aggr = 'sum'):
                super().__init__()
                if backbone == 'GCN':
                    self.conv1 = GCNConv(in_channels, hidden_channels, cached=True,
                                    normalize=True)
                    self.conv2 = GCNConv(hidden_channels, out_channels, cached=True,
                                    normalize=True)
                elif backbone == 'GAT':
                    self.conv1 = GATConv(in_channels, hidden_channels, heads)
                    self.conv2 = GATConv(hidden_channels, out_channels, heads)
                elif backbone == 'GraphSAGE':
                    self.conv1 = SAGEConv(in_channels, hidden_channels, aggr)
                    self.conv2 = SAGEConv(hidden_channels, out_channels, aggr)
                elif backbone == 'SGC':
                    self.conv1 = SGConv(in_channels, hidden_channels)
                    self.conv2 = SGConv(hidden_channels, out_channels)

            def forward(self, x, edge_index, edge_weight=None, return_h = False):
                x = F.dropout(x, p=0.5, training=self.training)
                x = self.conv1(x, edge_index, edge_weight).relu()
                if return_h:
                    return x
                x = F.dropout(x, p=0.5, training=self.training)
                x = self.conv2(x, edge_index, edge_weight)
                return x

      class GNN_Multi_Layer(torch.nn.Module):
          def __init__(self, in_channels, hidden_channels, out_channels, backbone, heads = 1, aggr = 'sum', num_layers = 2):
              super().__init__()

              self.convs = torch.nn.ModuleList()
              if num_layers == 1:
                  if backbone == 'GCN':
                      self.convs.append(GCNConv(in_channels, out_channels, cached=True, normalize=True))
                  elif backbone == 'GAT':
                      self.convs.append(GATConv(in_channels, out_channels, heads))
                  elif backbone == 'GraphSAGE':
                      self.convs.append(SAGEConv(in_channels, out_channels, aggr))
                  elif backbone == 'SGC':
                      self.convs.append(SGConv(in_channels, out_channels))
              else:
                  if backbone == 'GCN':
                      self.convs.append(GCNConv(in_channels, hidden_channels, cached=True, normalize=True))
                  elif backbone == 'GAT':
                      self.convs.append(GATConv(in_channels, hidden_channels, heads))
                  elif backbone == 'GraphSAGE':
                      self.convs.append(SAGEConv(in_channels, hidden_channels, aggr))
                  elif backbone == 'SGC':
                      self.convs.append(SGConv(in_channels, hidden_channels))
                  for _ in range(num_layers-2):
                      if backbone == 'GCN':
                          self.convs.append(GCNConv(hidden_channels, hidden_channels, cached=True, normalize=True))
                      elif backbone == 'GAT':
                          self.convs.append(GATConv(hidden_channels, hidden_channels, heads))
                      elif backbone == 'GraphSAGE':
                          self.convs.append(SAGEConv(hidden_channels, hidden_channels, aggr))
                      elif backbone == 'SGC':
                          self.convs.append(SGConv(hidden_channels, hidden_channels))
                  if backbone == 'GCN':
                      self.convs.append(GCNConv(hidden_channels, out_channels, cached=True, normalize=True))
                  elif backbone == 'GAT':
                      self.convs.append(GATConv(hidden_channels, out_channels, heads))
                  elif backbone == 'GraphSAGE':
                      self.convs.append(SAGEConv(hidden_channels, out_channels, aggr))
                  elif backbone == 'SGC':
                      self.convs.append(SGConv(hidden_channels, out_channels))
          def reset_parameters(self):

              for conv in self.convs:
                  conv.reset_parameters()

          def forward(self, x, adjs):
                  for idx, (edge_index, _, size) in enumerate(adjs):
                      if idx >= len(self.convs):
                          break

                      x_target = x[:size[1]]
                      x = self.convs[idx]((x, x_target), edge_index)

                      if idx != len(self.convs) - 1:
                          x = F.relu(x)
                          x = F.dropout(x, p=0.5, training=self.training)

                  return x
      #*********************************************
      #regression task
      #model = SAGERegressor(data.num_features, 3, 1, num_layers=3)
      model = GNN_Multi_Layer(in_channels = data.num_features,hidden_channels = 16,out_channels = 1,backbone = 'GraphSAGE')
      model.reset_parameters()
      model = model.to(device)
      #print(model)
      #*********************************************
      model.reset_parameters()
      #*********************************************
      optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)
      def train(epoch):
          model.train()
          total_loss = 0
          for batch_size, n_id, adjs in train_loader:
              adjs = [adj.to(device) for adj in adjs]
              optimizer.zero_grad()


              out = model(x[n_id], adjs)
              out = out[:batch_size, :]

              # The target tensor should match the batch size
              target = y[n_id[:batch_size]]

              loss = F.mse_loss(out, target.unsqueeze(1))
              loss.backward()
              optimizer.step()
              total_loss += loss.item()
          return total_loss



      for epoch in range(1,50):
          loss = train(epoch)
      #*********************************************
      Calib_loader = NeighborSampler(data.edge_index,node_idx = cali_mask
        , sizes = [3,3,3],batch_size = int((test_mask+cali_mask).sum()),shuffle=False)

      Test_loader = NeighborSampler(data.edge_index,node_idx = test_mask,sizes = [3,3,3],batch_size = int((test_mask+cali_mask).sum()),shuffle=False)
      #*********************************************
      #function the takes graph loader to the third layer embedding and the fn which is
      #a matrix in the form [x,predicted y]
      def inference_1(subgraph_loader):
          model.eval()
          all_c = []
          all_emb3 = []

          for batch_size, n_id, adjs in subgraph_loader:
              adjs = [adj.to(device) for adj in adjs]

              emb3 = model(x[n_id], adjs)[:batch_size]

              c = x[n_id][:batch_size]

              all_c.append(c)
              all_emb3.append(emb3)

          # Concatenate all the batches together
          all_c = torch.cat(all_c, dim=0)
          all_emb3 = torch.cat(all_emb3, dim=0)

          fn = torch.cat((all_c, all_emb3), dim=1)

          return fn

      #*********************************************

      fn_Calib = inference_1(Calib_loader)
      fn_Test = inference_1(Test_loader)
      fn_Calib = fn_Calib.detach().cpu().numpy()
      fn_Test = fn_Test.detach().cpu().numpy()
      fn = np.vstack((fn_Calib,fn_Test))
      prediction_dimension = data.num_features
      #*********************************************
      dimension = data.num_features
      #x is input vectors with each row representing one data and output is predicted label vector
      def get_xhat(x_vector):
        if(x_vector.shape[1]!=1):
              xhat_1= np.ones((x_vector.shape[0], 1))
              for j in range(x_vector.shape[0]):
                for i in range((cali_mask+test_mask).sum()):
                  if(np.array_equal(x_vector[j],fn[i,0:dimension])):
                    xhat_1[j]=fn[i,dimension]
              return xhat_1
        else:
              for i in range((cali_mask+test_mask).sum()):
                if(np.array_equal(x_vector.squeeze(),fn[i,0:dimension])):
                  return fn[i,dimension]
      #*********************************************
      #Two sided confidence interval
      #score_fn = lambda x, y : np.abs(y - get_xhat(x).squeeze())
      score_fn = lambda x, y : (y - get_xhat(x)).squeeze()
      score_inv_fn = lambda s, x : [get_xhat(x)-s, get_xhat(x) + s]
      score_inv_fn_lb = lambda s, x : [get_xhat(x) + s, np.inf]
      score_inv_fn_ub = lambda s, x : [-np.inf, get_xhat(x) - s]
      #*********************************************
      def indicator_matrix(scalar_values, disc):
          scalar_values = np.array(scalar_values)
          intervals = [(disc[i], disc[i + 1]) for i in range(len(disc) - 1)]
          matrix = np.zeros((scalar_values.shape[0], len(intervals)))
          for i in range(scalar_values.shape[0]):
            value = scalar_values[i,0]
            for j,(a,b) in enumerate(intervals):
              if (a<= value <b):
                matrix[i,j] = 1

          return matrix
      #*********************************************
      #Consider covariate shift of one of the dimension
      dimension_interest = 2
      #*********************************************
      minimum = data.x[:,dimension_interest].min()
      maximum = data.x[:,dimension_interest].max()
      #disc depends on the minimum and maximum of the second dimension
      #We next consider multiple types of covariate shifts and the last one is without any shifts
      disc = np.arange(math.ceil(minimum)-1, math.ceil(maximum),0.5)
      def phi_fn_groups(x_12):
        if len(x_12.shape) == 2:
          a=x_12[:x_12.shape[0],dimension_interest]
          a = a.reshape(-1,1)
          return indicator_matrix(a, disc)
        else:
          a = x_12.reshape(-1,1)
          a = a[dimension_interest]
          a = a.reshape(-1,1)
          return indicator_matrix(a, disc)

      #The gaussian density tilts with mean and variance
      eval_locs = [-1.5, -0.5]
      eval_scale = 0.2

      other_locs = [0, 1,1.5]
      other_scale = 1

      def phi_fn_shifts(x_13):
          if(len(x_13.shape) == 2):
            a = x_13[:x_13.shape[0],dimension_interest]

            shifts = [norm.pdf(a, loc=loc, scale=eval_scale).reshape(-1,1)
                          for loc in eval_locs]
            shifts.extend([norm.pdf(a, loc=loc, scale=other_scale).reshape(-1,1)
                          for loc in other_locs])
            shifts.append(np.ones((a.shape[0], 1)))
            return np.concatenate(shifts, axis=1)

          else:
            a = x_13.reshape(-1,1)
            a = a[dimension_interest]
            shifts = [norm.pdf(a, loc=loc, scale=eval_scale).reshape(-1,1)
                          for loc in eval_locs]
            shifts.extend([norm.pdf(a, loc=loc, scale=other_scale).reshape(-1,1)
                          for loc in other_locs])
            shifts.append(np.ones((a.shape[0], 1)))
            return np.concatenate(shifts, axis=1)
      def intercept(x_14):
        if(len(x_14.shape)==2):
          return x_14
        else:
          return x_14.reshape(1,data.num_features)
      """
      def phi_kernel_estimation(x_15):
        if(len(x_15.shape) == 2):
          print('2')
          ones_vector = np.ones((x_15.shape[0],1))
          new_data_point_std = x_15.reshape(x_15.shape[0],data.num_features)
          projected_vector = pca.transform(new_data_point_std)
          return np.column_stack((kde(projected_vector.T).reshape(x_15.shape[0],1),ones_vector))

        else:
          print('1')
          ones_vector = np.ones((1,1))
          new_data_point_std = x_15.reshape(1,data.num_features)
          projected_vector = pca.transform(new_data_point_std)

          return np.column_stack((kde(projected_vector).reshape(1,1),ones_vector))
      """

      def phi_kernel_estimation(x_15):

          if len(x_15.shape) == 2:  # Multiple data points
              kde_estimations = [kde(x_15[:, i]).reshape(-1, 1) for i, kde in enumerate(kdes)]
              ones_vector = np.ones((x_15.shape[0], 1))
              return np.hstack(kde_estimations + [ones_vector])
          else:  # Single data point
              kde_estimations = [kde([x_15[i]]).reshape(1, 1) for i, kde in enumerate(kdes)]
              ones_vector = np.array([[1]])
              return np.hstack(kde_estimations + [ones_vector])



      #*********************************************
      #Set up the problem by kernel density estimation
      infinite_params = {}
      cond_conf = CondConf(score_fn, phi_kernel_estimation, infinite_params)
      cond_conf.setup_problem(x_calib, y_calib)

      n_test = len(x_test)
      alpha = 0.05
      lbs_kernel = np.zeros((n_test,))
      ubs_kernel= np.zeros((n_test,))
      i = 0
      for x_t in tqdm(x_test):
          """
          res = cond_conf.predict(1-alpha, x_t, score_inv_fn)
          lbs_kernel[i] = res[0]
          ubs_kernel[i] = res[1]
          """
          res = cond_conf.predict(alpha / 2, x_t, score_inv_fn_lb, exact=True, randomize=True)
          lbs_kernel[i] = res[0]
          res = cond_conf.predict(1 - alpha / 2, x_t, score_inv_fn_ub, exact=True, randomize=True)
          ubs_kernel[i] = res[1]
          i += 1
      #*********************************************
      #Set up the problem by groups
      """
      infinite_params = {}
      #cond_conf = CondConf(score_fn, phi_fn_shifts, infinite_params)
      #cond_conf.setup_problem(x_calib, y_calib)

      n_test = len(x_test)
      alpha = 0.05
      lbs_groups = np.zeros((n_test,))
      ubs_groups = np.zeros((n_test,))
      i = 0
      for x_t in tqdm(x_test):
          res = cond_conf.predict(1-alpha, x_t, score_inv_fn)
          lbs_groups[i] = res[0]
          ubs_groups[i] = res[1]
        i += 1
      """

      def get_marginalcoverage_condi(ubs_1,lbs_1):
        j=0
        for i in range(len(x_test)):
          if((y_test[i]<=ubs_1[i]) & (y_test[i]>=lbs_1[i])):
            j+=1

        return j/len(x_test)
      def get_marginalcoverage_split():
        j=0
        x_hat = get_xhat(x_test)
        for i in range(len(x_test)):
          if((y_test[i]>=x_hat[i]-q )& (y_test[i]<= x_hat[i]+q)):
            j+=1

        return j/len(x_test)
      def evaluate_predictions(S, X, y):
          marg_coverage = np.mean([y[i] in S[i] for i in range(len(y))])
          wsc_coverage = wsc_unbiased(X, y, S,delta=0.1,M=1000)
          #print('Marginal coverage:       {:2.3%}'.format(marg_coverage))
          #print('WS conditional coverage: {:2.3%}'.format(wsc_coverage))
          return wsc_coverage
      q = np.quantile(np.abs(get_xhat(x_calib) - y_calib.reshape(-1,1)),
                      np.ceil((len(x_calib) + 1) * (0.95)) / len(x_calib),)
      """
      marginal_coverage_split = get_marginalcoverage_split()
      marginal_coverage_condi_kernel = get_marginalcoverage_condi(ubs_kernel,lbs_kernel)
      marginal_condi.append(marginal_coverage_condi_kernel)
      marginal_split.append(marginal_coverage_split)
      ubs_condi_kernel.append(ubs_kernel)
      lbs_condi_kernel.append(lbs_kernel)
      ubs_split.append(get_xhat(x_test)+q)
      lbs_split.append(get_xhat(x_test)-q)
      """
      #Check the marginal and WS conditional coverage
      ubs = (get_xhat(x_test)+q).squeeze()
      lbs =(get_xhat(x_test)-q).squeeze()
      test_indices = test_mask
      intervals = [pd.Interval(left=lb, right=ub, closed='right') for lb, ub in zip(lbs, ubs)]
      print(f"The empirical marginal and WS conditional coverage using split conformal is: ")
      print(evaluate_predictions(intervals,x[test_indices].detach().cpu().numpy(),y[test_indices].detach().cpu().numpy()))
      marg_coverage = np.mean([y[test_indices].detach().cpu().numpy()[i] in intervals[i] for i in range(len(y[test_indices].detach().cpu().numpy()))])
      wsc_coverage = evaluate_predictions(intervals,x[test_indices].detach().cpu().numpy(),y[test_indices].detach().cpu().numpy())
      split_wsc_condition.append(evaluate_predictions(intervals,x[test_indices].detach().cpu().numpy(),y[test_indices].detach().cpu().numpy()))
      split_length.append(2*q)
      current_test_features = x[test_mask].detach().cpu().numpy()
      coverage_flag = [y[test_indices].detach().cpu().numpy()[i] in intervals[i] for i in range(len(y[test_indices].detach().cpu().numpy()))]

      dataset_results[dataset].get('method_split_conformal')[f'Iteration_{c}'] = {
          'Marginal_Coverage': marg_coverage,
          'Conditional_Coverage': wsc_coverage,
          'test_mask':test_mask,
          'coverage_flag':coverage_flag,
          'Average length':2*q,
          'ubs':ubs,
          'lbs':lbs}

      ubs = ubs_kernel
      lbs = lbs_kernel
      test_indices = test_mask
      intervals = [pd.Interval(left=lb, right=ub, closed='right') for lb, ub in zip(lbs, ubs)]
      #print(f"The empirical marginal and WS conditional coverage using kernel density is: ")
      print(marginal_coverage)
      print(evaluate_predictions(intervals,x[test_indices].detach().cpu().numpy(),y[test_indices].detach().cpu().numpy()))
      #condition_calibrated_marginal.append(get_marginalcoverage_condi(ubs,lbs))
      #condition_calibrated_wsc_condition.append(evaluate_predictions(intervals,x[test_indices].detach().cpu().numpy(),y[test_indices].detach().cpu().numpy()))
      #condition_calibrated_length.append(ubs-lbs)
      marginal_coverage = np.mean([y[test_indices].detach().cpu().numpy()[i] in intervals[i] for i in range(len(y[test_indices].detach().cpu().numpy()))])
      wsc_coverage = evaluate_predictions(intervals,x[test_indices].detach().cpu().numpy(),y[test_indices].detach().cpu().numpy())
      coverage_flag = [y[test_indices].detach().cpu().numpy()[i] in intervals[i] for i in range(len(y[test_indices].detach().cpu().numpy()))]
      average_length = np.mean(ubs-lbs)
      #print(wsc_coverage)
      #print(average_length)
      dataset_results[dataset].get('method_condition_calibrated')[f'Iteration_{c}'] = {
          'Marginal_Coverage': marginal_coverage ,
          'Conditional_Coverage': wsc_coverage,
          'test_mask':test_mask,
          'coverage_flag':coverage_flag,
          'Average length':average_length,
          'ubs':ubs,
          'lbs':lbs}
      #print(c)
import pickle
file_path = '/content/drive/MyDrive/Regression_SplitwithCondi_experiment_results.txt'
with open(file_path, 'wb') as file:
    pickle.dump(dataset_results, file)




Anaheim
dataset name Anaheim
Data(x=[914, 4], edge_index=[2, 3870], y=[914])
Not Chicago Sketch


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.NeighborSampler' is deprecated, use 'loader.NeighborSampler' instead
  warnings.warn(out)


ValueError: Cannot broadcast dimensions  (276,) (75626,)

Run their regression model and collect the results

In [38]:
score_fn(x_calib, y_calib).shape

(275, 275)

In [40]:
phi_kernel_estimation(x_calib).shape

(275, 5)

In [ ]:
datasets = ['Anaheim','ChicagoSketch','county_election_2016','county_education_2012','county_income_2012','county_unemployment_2012','twitch_PTBR']
dataset_results = {dataset: {} for dataset in datasets}
for dataset_name in datasets:
      marginal_coverage_all = []
      conditional_coverage_all = []
      for z in range(50):
          !python train.py --model GraphSAGE \
                      --dataset {dataset_name} \
                      --device cuda \
                      --alpha 0.05 \
                      --num_runs 1 \
                      --conf_correct_model QR \
                      --data_seed {z+100}

          import pickle
          file_path = '/content/conformalized-gnn-master/data_splits.pkl'
          with open(file_path, 'rb') as file:
            data_splits = pickle.load(file)
          file_path = f'/content/conformalized-gnn-master/pred/{dataset_name}_GraphSAGE_QR_alpha_0.05.pkl'
          with open(file_path, 'rb') as file:
            results1 = pickle.load(file)
          file_path = '/content/conformalized-gnn-master/labelsandfeatures.pkl'
          with open(file_path, 'rb') as file:
            data_labels = pickle.load(file)

          #print(data_labels.get('labels')[~results1[0].get('conf_gnn').get('Raw')[4][0]])
          current_data_labels = data_labels.get('labels')
          current_data_features = data_labels.get('features')

          current_calib_test_mask = data_labels.get('calib_test_mask')
          #print(current_calib_test_mask.shape)
          current_test_mask_outof_calibtest = ~current_calib_test_mask
          current_test_features = current_data_features[current_calib_test_mask][~results1[0].get('gnn').get('CQR')[4][0]]
          current_test_labels = current_data_labels[~results1[0].get('gnn').get('CQR')[4][0]]
          current_lbs = results1[0].get('gnn').get('CQR')[2][0][0]
          current_ubs = results1[0].get('gnn').get('CQR')[2][0][1]
          average_length = np.mean(current_ubs - current_lbs)
          intervals = [pd.Interval(left=lb, right=ub, closed='right') for lb, ub in zip(current_lbs, current_ubs)]
          marg_coverage = np.mean([current_test_labels[i] in intervals[i] for i in range(len(current_test_labels))])
          coverage_flag = [current_test_labels[i] in intervals[i] for i in range(len(current_test_labels))]
          #wsc_coverage = coverage.wsc_unbiased(current_test_features, current_test_labels, intervals,delta=0.05,M=5000)
          dataset_results[dataset_name][f'Iteration_{z}'] = {
            'Marginal_Coverage': marg_coverage,
            'current_test_features':current_test_features,
            'coverage_flag':coverage_flag,
            'Average length':average_length,
            'current_data_features':current_data_features,
            'current_calib_test_mask':current_calib_test_mask,
            'current_test_mask_outof_calibtest':current_test_mask_outof_calibtest,
            'current_test_labels':current_test_labels}


# Save the entire dictionary to a pickle file
file_path = '/content/drive/MyDrive/Regression_GNN_Multi_results.txt'
with open(file_path, 'wb') as file:
    pickle.dump(dataset_results, file)

Run their classification model and collect the results

In [ ]:
import tqdm
datasets = ['Cora_ML_CF','DBLP_CF','CiteSeer_CF','PubMed_CF','Amazon-Computers','Amazon-Photo','Coauthor-CS','Coauthor-Physics']
dataset_results = {dataset: {} for dataset in datasets}
for dataset_name in datasets:
      marginal_coverage_all = []
      conditional_coverage_all = []
      for z in range(20):
          !python train.py --model GraphSAGE \
                      --dataset {dataset_name} \
                      --device cpu \
                      --alpha 0.05 \
                      --num_runs 1 \
                      --conf_correct_model Calibrate \
                      --calibrator TS \
                      --data_seed {z+200}

          import pickle
          file_path = '/content/conformalized-gnn-master/data_splits.pkl'
          with open(file_path, 'rb') as file:
            data_splits = pickle.load(file)
          file_path = f'/content/conformalized-gnn-master/pred/{dataset_name}_GraphSAGE_TS_alpha_0.05.pkl'
          with open(file_path, 'rb') as file:
            results1 = pickle.load(file)
          file_path = '/content/conformalized-gnn-master/labelsandfeatures.pkl'
          with open(file_path, 'rb') as file:
            data_labels = pickle.load(file)
          import numpy as np
          #print(data_labels.get('labels')[~results1[0].get('conf_gnn').get('Raw')[4][0]])
          current_data_labels = data_labels.get('labels')
          current_data_features = data_labels.get('features')
          current_calib_test_mask = data_labels.get('calib_test_mask')
          #print(current_calib_test_mask.shape)
          current_test_mask_outof_calibtest = ~current_calib_test_mask
          current_test_features = current_data_features[current_calib_test_mask][~results1[0].get('gnn').get('APS')[4][0]]
          current_test_labels = current_data_labels[~results1[0].get('gnn').get('APS')[4][0]]
          current_predictions = results1[0].get('gnn').get('APS')[2][0]
          true_indices_per_row = [list(np.where(row)[0]) for row in current_predictions]
          avg_interval_length = np.mean([len(row) for row in true_indices_per_row])
          marg_coverage = np.mean([current_test_labels[i] in true_indices_per_row[i] for i in range(len(current_test_labels))])
          coverage_flag = [current_test_labels[i] in true_indices_per_row[i] for i in range(len(current_test_labels))]
          #print([current_test_labels[i] in true_indices_per_row[i] for i in range(len(current_test_labels))])
          wsc_coverage = coverage.wsc_unbiased(current_test_features, current_test_labels, true_indices_per_row,delta=0.1,M=1000)
          dataset_results[dataset_name][f'Iteration_{z}'] = {
            'Marginal_Coverage': marg_coverage,
            'Conditional_Coverage':wsc_coverage,
            'Average_Interval_Length': avg_interval_length,
            'current_test_features':current_test_features,
            'coverage_flag':coverage_flag,
            'current_data_features':current_data_features,
            'current_calib_test_mask':current_calib_test_mask,
            'current_test_mask_outof_calibtest':current_test_mask_outof_calibtest}

file_path = '/content/drive/MyDrive/Classification_GNN_results.txt'
import pickle
# Save the final results to a file
with open(file_path, 'wb') as file:
    pickle.dump(dataset_results, file)



In [ ]:
file_path = '/content/drive/MyDrive/Regression_CFGNN_results.txt'
with open(file_path, 'rb') as file:
    loaded_results = pickle.load(file)

for dataset_name in dataset_results:
    marginal_coverage_sum = 0
    conditional_coverage_sum = 0
    count_valid_conditional = 0

    for i in range(30):
        iteration_key = f'Iteration_{i}'
        marginal_coverage_sum += dataset_results[dataset_name][iteration_key]['Marginal_Coverage']

        conditional_coverage = dataset_results[dataset_name][iteration_key]['Conditional_Coverage']
        if not np.isnan(conditional_coverage):  # Check if conditional coverage is not nan
            conditional_coverage_sum += conditional_coverage
            count_valid_conditional += 1

    avg_marginal_coverage = marginal_coverage_sum / 30
    avg_conditional_coverage = conditional_coverage_sum / count_valid_conditional if count_valid_conditional > 0 else 'nan'

    print(f'{dataset_name}:')
    print(f'  Average Marginal Coverage: {avg_marginal_coverage}')
    print(f'  Average Conditional Coverage: {avg_conditional_coverage}\n')

possible plot to see coverage

In [ ]:
#Plot the first result of the dimension of interest of covariates
x_test = x[test_mask_lists[0]]
y_test = y[test_mask_lists[0]]
ubs = ubs_condi_kernel[4]
lbs = lbs_condi_kernel[4]
dimension_interest = 1

cp = sns.color_palette()
sns.set(font="DejaVu Sans")
sns.set_style("whitegrid", {'axes.grid' : False})
fig = plt.figure()
fig.set_size_inches(17.5, 6)
sort_order = np.argsort(x_test[0:n_test,dimension_interest])
x_test_s = x_test[0:n_test,dimension_interest][sort_order]
y_test_s = y_test[sort_order]
y_test_hat = get_xhat(x_test[sort_order])
lb = lbs[sort_order]
ub = ubs[sort_order]

ax1 = fig.add_subplot(1, 3, 1)
ax1.plot(x_test_s, y_test_s, '.', alpha=0.2)
ax1.plot(x_test_s, y_test_hat, lw=1, color='k')
ax1.plot(x_test_s, y_test_hat + q, color=cp[0], lw=2)
ax1.plot(x_test_s, y_test_hat - q, color=cp[0], lw=2)

ax1.set_ylim(-5,10)
ax1.tick_params(axis='both', which='major', labelsize=14)
ax1.set_xlabel("$X$", fontsize=16, labelpad=10)
ax1.set_ylabel("$Y$", fontsize=16, labelpad=10)
ax1.set_title("Split Conformal", fontsize=18, pad=12)
ax1.fill_between(x_test_s.flatten(), y_test_hat.flatten() - q, y_test_hat.flatten() + q,
                 color=cp[3], alpha=0.4, label='split prediction interval')


ax2 = fig.add_subplot(1, 3, 2, sharex = ax1, sharey = ax1)
ax2.plot(x_test_s, y_test_s, '.', alpha=0.2)
ax2.plot(x_test_s, y_test_hat, color='k', lw=1)
ax2.plot(x_test_s, ub, color=cp[1], lw=2)
ax2.plot(x_test_s, lb, color=cp[1], lw=2)
ax2.fill_between(x_test_s.flatten(), lb,
                 ub,
                 color=cp[3], alpha=0.4, label='conditional calibration')
ax2.tick_params(axis='both', which='major', direction='out', labelsize=14)
ax2.set_xlabel("$X$", fontsize=16, labelpad=10)
ax2.set_ylabel("$Y$", fontsize=16, labelpad=10)
ax2.set_title("Conditional Calibration", fontsize=18, pad=12)


